## 1.1 Getting Data

Let us consider a public database, called “Adult” dataset hosted on the UCI’s Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/Adult), that contains approximately 32.000 observations about different financial parameters of US population.

In [43]:
import pandas as pd
import numpy as np
%matplotlib inline

In [44]:
data= pd.read_csv('adult.csv')

In [45]:
# check total nan values in the dataset

for column in data.columns:
    print(column + ":", sum(data[column] == "?"))

age: 0
workclass: 2799
fnlwgt: 0
education: 0
educational-num: 0
marital-status: 0
occupation: 2809
relationship: 0
race: 0
gender: 0
capital-gain: 0
capital-loss: 0
hours-per-week: 0
native-country: 857
income: 0


### 1.4 Data Cleaning

The most common steps are:

+ **Sample the data**. If the amount of raw data is huge, processing all of them may require an extensive amount of processing power which may not be practical.  In this case, it is quite common to sample the input data to reduce the size of data that need to be processed.

+ **Impute missing data**. It is quite common that some of the input records are incomplete in the sense that certain fields are missing or have input error.  In a typical tabular data format, we need to validate each record contains the same number of fields and each field contains the data type we expect. In case the record has some fields missing, we have the following choices: 
<small>
* (a) Discard the whole record if it is incomplete; 
* (b) Infer the missing value based on the data from other records.  A common approach is to fill the missing data with the average, or the median.
<small>

In [46]:
import numpy as np
import pandas as pd

# Sample the data into n equal parts. After sampling, replace "?" with NaN
data = data.sample(frac=0.5)
data = data.replace('?', np.nan)

# Replace NaN with mean for numerical columns where "?" was replaced
for column in data.columns:
    if data[column].dtype != 'object':  # Check if column is numerical
        data[column] = pd.to_numeric(data[column])  # Convert to numeric
        mean_value = data[column].mean()
        data[column] = data[column].fillna(mean_value)



In [47]:
# count total "?" in each column and display each columns total nulls, use for loops and if else

for column in data.columns:
    print(column + ":", sum(data[column] == "?"))


age: 0
workclass: 0
fnlwgt: 0
education: 0
educational-num: 0
marital-status: 0
occupation: 0
relationship: 0
race: 0
gender: 0
capital-gain: 0
capital-loss: 0
hours-per-week: 0
native-country: 0
income: 0


+ **Normalize numeric value**. Normalize data is about transforming numeric data into a uniform range.

In [49]:
numeric_columns = data.select_dtypes(include=['int', 'float']).columns
data[numeric_columns] = data[numeric_columns].apply(pd.to_numeric)

# Min-Max scaling
data_normalized = (data[numeric_columns] - data[numeric_columns].min()) / (data[numeric_columns].max() - data[numeric_columns].min())

# Display the normalized data
print(data_normalized.head())

            age    fnlwgt  educational-num  capital-gain  capital-loss  \
18140  0.410959  0.107441         0.733333           0.0           0.0   
7420   0.369863  0.018766         0.533333           0.0           0.0   
1845   0.041096  0.143388         0.600000           0.0           0.0   
10960  0.246575  0.005428         0.600000           0.0           0.0   
33068  0.000000  0.179165         0.400000           0.0           0.0   

       hours-per-week  
18140        0.346939  
7420         0.397959  
1845         0.346939  
10960        0.397959  
33068        0.295918  


+ **Reduce dimensionality**. High dimensionality can be a problem for some machine learning methods.  There are two ways to reduce the number of input features.  One is about $removing$ $irrelevant$ input variables, another one is about $removing$ $redundant$ input variables.

In [50]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)  # You can specify the number of components
data_reduced = pca.fit_transform(data_normalized)

# Create a DataFrame with reduced dimensions
data_reduced_df = pd.DataFrame(data_reduced, columns=['PC1', 'PC2'])

# Display the reduced data
print(data_reduced_df.head())

        PC1       PC2
0  0.131945 -0.082138
1  0.057992  0.086781
2 -0.255223 -0.045998
3 -0.045885 -0.006842
4 -0.346307  0.143557


+ **Add derived features**. In some cases, we may need to compute additional attributes from existing attributes (f.e. converting a geo-location to a zip code, or converting the age to an age group).

In [51]:
# Derive Age Group
def get_age_group(age):
    if age < 18:
        return 'Under 18'
    elif age < 30:
        return '18-29'
    elif age < 40:
        return '30-39'
    elif age < 50:
        return '40-49'
    else:
        return '50 and above'

data['age_group'] = data['age'].apply(get_age_group)

# Derive Education Level Category
def get_education_level_category(education):
    if 'Bachelor' in education:
        return "Bachelor's Degree or Higher"
    elif 'Masters' in education or 'Doctorate' in education:
        return "Master's Degree or Higher"
    elif 'Assoc' in education:
        return "Associate's Degree"
    else:
        return 'Other'

data['education_level_category'] = data['education'].apply(get_education_level_category)


marital_status_mapping = {
    'Married-civ-spouse': 'Married',
    'Married-AF-spouse': 'Married',
    'Married-spouse-absent': 'Married',
    'Separated': 'Separated',
    'Divorced': 'Divorced',
    'Widowed': 'Widowed',
    'Never-married': 'Single'
}
data['marital_status_category'] = data['marital-status'].map(marital_status_mapping)

# Display the DataFrame with derived features
print(data.head())


       age  workclass  fnlwgt     education  educational-num  \
18140   47  Local-gov  171095    Assoc-acdm               12   
7420    44    Private   40024       HS-grad                9   
1845    20  Local-gov  224229  Some-college               10   
10960   35    Private   20308  Some-college               10   
33068   17    Private  277112          11th                7   

           marital-status       occupation   relationship   race  gender  \
18140  Married-civ-spouse     Adm-clerical           Wife  White  Female   
7420   Married-civ-spouse     Craft-repair        Husband  White    Male   
1845        Never-married  Exec-managerial  Not-in-family  White  Female   
10960           Separated            Sales  Not-in-family  White    Male   
33068       Never-married            Sales      Own-child  White    Male   

       capital-gain  capital-loss  hours-per-week native-country income  \
18140             0             0              35  United-States   >50K   
7420    


+ **Discretize numeric value into categories**. Discretize data is about cutting a continuous value into ranges and assigning the numeric with the corresponding bucket of the range it falls on.  For numeric attribute, a common way to generalize it is to discretize it into ranges, which can be either constant width (variable height/frequency) or variable width (constant height).

In [52]:
import pandas as pd


num_bins = 5  # You can adjust the number of bins as needed
bin_edges = [0, 20, 40, 60, 80, 100]

bin_labels = ['0-20', '21-40', '41-60', '61-80', '81-100']

data['age_category'] = pd.cut(data['age'], bins=bin_edges, labels=bin_labels, include_lowest=True)

print(data[['age', 'age_category']].head())

       age age_category
18140   47        41-60
7420    44        41-60
1845    20         0-20
10960   35        21-40
33068   17         0-20



+ **Binarize categorical attributes**. Certain machine learning models only take binary input (or numeric input).  In this case, we need to convert categorical attribute into multiple binary attributes, while each binary attribute corresponds to a particular value of the category. 

In [53]:
import pandas as pd

categorical_columns = data.select_dtypes(include=['object']).columns

data_encoded = pd.get_dummies(data, columns=categorical_columns)

# Display the DataFrame with one-hot encoded columns
print(data_encoded.head())


       age  fnlwgt  educational-num  capital-gain  capital-loss  \
18140   47  171095               12             0             0   
7420    44   40024                9             0             0   
1845    20  224229               10             0             0   
10960   35   20308               10             0             0   
33068   17  277112                7             0             0   

       hours-per-week age_category  workclass_Federal-gov  \
18140              35        41-60                      0   
7420               40        41-60                      0   
1845               35         0-20                      0   
10960              40        21-40                      0   
33068              30         0-20                      0   

       workclass_Local-gov  workclass_Never-worked  ...  age_group_Under 18  \
18140                    1                       0  ...                   0   
7420                     0                       0  ...                 

+ **Select, combine, aggregate data**. Designing the form of training data is the most important part of the whole predictive modeling exercise because the accuracy largely depends on whether the input features are structured in an appropriate form that provide strong signals to the learning algorithm. Rather than using the raw data as it is, it is quite common that multiple pieces of raw data need to be combined together, or aggregating multiple raw data records along some dimensions.

In [54]:
import pandas as pd

selected_features = ['age', 'education', 'marital-status', 'occupation', 'relationship', 'income']
selected_data = data[selected_features]

data['is_married'] = (data['marital-status'] == 'Married-civ-spouse') | (data['marital-status'] == 'Married-AF-spouse')

aggregated_data = data.groupby('education')['hours-per-week'].mean().reset_index()
aggregated_data.columns = ['education', 'avg_hours_per_week']

# Display the processed data
print(selected_data.head())
print(aggregated_data.head())


       age     education      marital-status       occupation   relationship  \
18140   47    Assoc-acdm  Married-civ-spouse     Adm-clerical           Wife   
7420    44       HS-grad  Married-civ-spouse     Craft-repair        Husband   
1845    20  Some-college       Never-married  Exec-managerial  Not-in-family   
10960   35  Some-college           Separated            Sales  Not-in-family   
33068   17          11th       Never-married            Sales      Own-child   

      income  
18140   >50K  
7420   <=50K  
1845   <=50K  
10960  <=50K  
33068  <=50K  
  education  avg_hours_per_week
0      10th           36.791608
1      11th           33.937155
2      12th           35.114551
3   1st-4th           38.442857
4   5th-6th           39.423077
